# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成計數編碼以及搭配邏輯斯迴歸的預測 (In[4], Out[4], In[5], Out[5]) 
- 仿造範例, 完成雜湊編碼, 以及計數編碼+雜湊編碼 搭配邏輯斯迴歸的預測 (In[6], Out[6], In[7], Out[7]) 
- 試著回答上述執行結果的觀察

# 作業1
* 參考範例，將鐵達尼的艙位代碼( 'Cabin' )欄位使用特徵雜湊 / 標籤編碼 / 計數編碼三種轉換後，  
與其他類別型欄位一起預估生存機率

In [185]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
import warnings; warnings.simplefilter('ignore')

data_path = 'data/'
df_train = pd.read_csv(data_path + 'titanic_train.csv')
df_test = pd.read_csv(data_path + 'titanic_test.csv')

train_Y = df_train['Survived']
ids = df_test['PassengerId']
df_train = df_train.drop(['PassengerId', 'Survived'] , axis=1)
df_test = df_test.drop(['PassengerId'] , axis=1)
df = pd.concat([df_train,df_test])
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [186]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'object':
        object_features.append(feature)
print(f'{len(object_features)} Object Features : {object_features}\n')

num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

df_obj = df[object_features].fillna('None')
df_num = df[num_features].fillna(0)
df = pd.concat([df_obj,df_num],axis=1)
train_num = train_Y.shape[0]
df.head()

5 Object Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

5 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S,3,22.0,1,0,7.2500
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C,1,38.0,1,0,71.2833
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S,3,26.0,0,0,7.9250
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S,1,35.0,1,0,53.1000
4,"Allen, Mr. William Henry",male,373450,None,S,3,35.0,0,0,8.0500


# 作業2
* 承上題，三者比較效果何者最好?
> 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸，但是微乎其微幾乎沒有改善

In [187]:
# 對照組 : 標籤編碼 + 邏輯斯迴歸
df_temp = pd.DataFrame()
for c in object_features:
    df_temp[c] = LabelEncoder().fit_transform(df[c])
for c in num_features:    
    df_temp[c] = MinMaxScaler().fit_transform(df[[c]])
train_X = df_temp[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_temp.head().shape

0.7901112743622125


(5, 10)

In [188]:
# 加上 'Cabin' 欄位的計數編碼
count_df = df.groupby(['Cabin'])['Name'].agg(Cabin_Count='size').reset_index()
df = pd.merge(df, count_df, on=['Cabin'], how='left')
count_df.sort_values(by=['Cabin_Count'], ascending=False).head(10)

,Cabin,Cabin_Count
185,None,1014
80,C23 C25 C27,6
184,G6,5
47,B57 B59 B63 B66,5
60,B96 B98,4
180,F2,4
117,D,4
79,C22 C26,4
181,F33,4
183,F4,4


In [189]:
# 'Cabin'計數編碼 + 邏輯斯迴歸
df_t = df_temp.copy()
df_t['Cabin_Count'] = df['Cabin_Count']
train_X = df_t[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_t.head()

0.8024710377060394


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_Count
0,155,1,720,185,3,1.0,0.2750,0.125,0.0,0.014151,1014
1,286,0,816,106,0,0.0,0.4750,0.125,0.0,0.139136,2
2,523,0,914,185,3,1.0,0.3250,0.000,0.0,0.015469,1014
3,422,0,65,70,3,0.0,0.4375,0.125,0.0,0.103644,2
4,22,1,649,185,3,1.0,0.4375,0.000,0.0,0.015713,1014


In [190]:
# 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_t = df_temp.copy()
df_t['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 8)
train_X = df_t[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_t.head()

0.7968277391164651


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_Hash
0,155,1,720,185,3,1.0,0.2750,0.125,0.0,0.014151,0
1,286,0,816,106,0,0.0,0.4750,0.125,0.0,0.139136,2
2,523,0,914,185,3,1.0,0.3250,0.000,0.0,0.015469,0
3,422,0,65,70,3,0.0,0.4375,0.125,0.0,0.103644,7
4,22,1,649,185,3,1.0,0.4375,0.000,0.0,0.015713,0


In [191]:
# 'Cabin'計數編碼 + 'Cabin'特徵雜湊 + 邏輯斯迴歸
df_t = df_temp.copy()
df_t['Cabin_Hash'] = df['Cabin'].map(lambda x:hash(x) % 16)
df_t['Cabin_Count'] = df['Cabin_Count']
train_X = df_t[:train_num]
estimator = LogisticRegression()
print(cross_val_score(estimator, train_X, train_Y, cv=5).mean())
df_t.head()

0.8069341053065356


,Name,Sex,Ticket,Cabin,Embarked,Pclass,Age,SibSp,Parch,Fare,Cabin_Hash,Cabin_Count
0,155,1,720,185,3,1.0,0.2750,0.125,0.0,0.014151,8,1014
1,286,0,816,106,0,0.0,0.4750,0.125,0.0,0.139136,2,2
2,523,0,914,185,3,1.0,0.3250,0.000,0.0,0.015469,8,1014
3,422,0,65,70,3,0.0,0.4375,0.125,0.0,0.103644,15,2
4,22,1,649,185,3,1.0,0.4375,0.000,0.0,0.015713,8,1014
